In [1]:
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [2]:
filepath = '../Resources/Age_Datasets/gen_Y_stroke.csv'
medical_df = pd.read_csv(filepath)
medical_df.head(10)

,ID#,Gender,Age,Hypertension,Heart_Disease,Ever_Married,Work_Type,Residence_Type,Avg_Glucose_Lvl,BMI,Smoker,Stroke
0,50,Female,39,1,0,Yes,Private,Rural,58.09,39.2,Current,1
1,119,Female,38,0,0,No,Self-employed,Urban,82.28,24.0,Former,1
2,134,Female,38,0,0,Yes,Private,Rural,101.45,NaN,Former,1
3,183,Female,32,0,0,Yes,Private,Rural,76.13,29.9,Current,1
4,211,Female,39,0,0,Yes,Private,Urban,83.24,26.3,Never,1
5,229,Female,39,0,0,Yes,Self-employed,Urban,97.76,29.6,Current,1
6,258,Female,32,0,0,Yes,Private,Rural,77.67,32.3,Current,0
7,262,Female,37,0,0,Yes,Private,Rural,162.96,39.4,Never,0
8,263,Female,37,0,0,Yes,Private,Rural,73.50,26.1,Former,0
9,264,Female,40,0,0,Yes,Private,Rural,95.04,42.4,Never,0


In [3]:
# Drop ID# column
medical_df.drop(columns=['ID#'], inplace=True)
medical_df.head()

,Gender,Age,Hypertension,Heart_Disease,Ever_Married,Work_Type,Residence_Type,Avg_Glucose_Lvl,BMI,Smoker,Stroke
0,Female,39,1,0,Yes,Private,Rural,58.09,39.2,Current,1
1,Female,38,0,0,No,Self-employed,Urban,82.28,24.0,Former,1
2,Female,38,0,0,Yes,Private,Rural,101.45,NaN,Former,1
3,Female,32,0,0,Yes,Private,Rural,76.13,29.9,Current,1
4,Female,39,0,0,Yes,Private,Urban,83.24,26.3,Never,1


In [29]:
medical_df['Stroke'].value_counts()

0    1212
1       6
Name: Stroke, dtype: int64

In [4]:
# Get a list of categorical columns
categorical_columns = medical_df.dtypes[medical_df.dtypes=='object'].index.tolist()
categorical_columns

['Gender', 'Ever_Married', 'Work_Type', 'Residence_Type', 'Smoker']

In [5]:
# Create OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit & transform OneHotEncoder using categorical columns
encode_df = pd.DataFrame(enc.fit_transform(medical_df[categorical_columns]))

# Add column names
encode_df.columns = enc.get_feature_names(categorical_columns)
print(encode_df.shape)
encode_df.head(10)

(1218, 14)


C:\Users\remem\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Never_worked,Work_Type_Private,Work_Type_Self-employed,Residence_Type_Rural,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
6,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
8,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
9,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [6]:
# Get encode_df info
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender_Female            1218 non-null   float64
 1   Gender_Male              1218 non-null   float64
 2   Ever_Married_No          1218 non-null   float64
 3   Ever_Married_Yes         1218 non-null   float64
 4   Work_Type_Govt_job       1218 non-null   float64
 5   Work_Type_Never_worked   1218 non-null   float64
 6   Work_Type_Private        1218 non-null   float64
 7   Work_Type_Self-employed  1218 non-null   float64
 8   Residence_Type_Rural     1218 non-null   float64
 9   Residence_Type_Urban     1218 non-null   float64
 10  Smoker_Current           1218 non-null   float64
 11  Smoker_Former            1218 non-null   float64
 12  Smoker_Never             1218 non-null   float64
 13  Smoker_Unknown           1218 non-null   float64
dtypes: float64(14)
memory us

In [7]:
# Drop redundant columns
encode_df.drop(columns=['Gender_Female', 'Ever_Married_No', 'Residence_Type_Rural'], inplace=True)
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender_Male              1218 non-null   float64
 1   Ever_Married_Yes         1218 non-null   float64
 2   Work_Type_Govt_job       1218 non-null   float64
 3   Work_Type_Never_worked   1218 non-null   float64
 4   Work_Type_Private        1218 non-null   float64
 5   Work_Type_Self-employed  1218 non-null   float64
 6   Residence_Type_Urban     1218 non-null   float64
 7   Smoker_Current           1218 non-null   float64
 8   Smoker_Former            1218 non-null   float64
 9   Smoker_Never             1218 non-null   float64
 10  Smoker_Unknown           1218 non-null   float64
dtypes: float64(11)
memory usage: 104.8 KB


In [8]:
# Merge encoded df with medical df
medical_df = medical_df.merge(encode_df, left_index=True, right_index=True).drop(categorical_columns, axis=1)
print(medical_df.shape)
medical_df.head(10)

(1218, 17)


,Age,Hypertension,Heart_Disease,Avg_Glucose_Lvl,BMI,Stroke,Gender_Male,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Never_worked,Work_Type_Private,Work_Type_Self-employed,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,39,1,0,58.09,39.2,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,38,0,0,82.28,24.0,1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2,38,0,0,101.45,NaN,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,32,0,0,76.13,29.9,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,39,0,0,83.24,26.3,1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
5,39,0,0,97.76,29.6,1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
6,32,0,0,77.67,32.3,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7,37,0,0,162.96,39.4,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8,37,0,0,73.50,26.1,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9,40,0,0,95.04,42.4,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Create SimpleImputer instance to replace missing BMI feature values with median BMI
imputer = SimpleImputer(strategy='mean')

# Fit SimpleImputer & transform data
med_transformed = imputer.fit_transform(medical_df)

In [10]:
# Add SimpleImputer outcome to dataframe
med_df_transformed = pd.DataFrame(med_transformed, columns=medical_df.columns)
print(med_df_transformed.shape)
med_df_transformed.head(10)

(1218, 17)


,Age,Hypertension,Heart_Disease,Avg_Glucose_Lvl,BMI,Stroke,Gender_Male,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Never_worked,Work_Type_Private,Work_Type_Self-employed,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,39.0,1.0,0.0,58.09,39.200000,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,38.0,0.0,0.0,82.28,24.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2,38.0,0.0,0.0,101.45,30.007608,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,32.0,0.0,0.0,76.13,29.900000,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,39.0,0.0,0.0,83.24,26.300000,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
5,39.0,0.0,0.0,97.76,29.600000,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
6,32.0,0.0,0.0,77.67,32.300000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7,37.0,0.0,0.0,162.96,39.400000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8,37.0,0.0,0.0,73.50,26.100000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9,40.0,0.0,0.0,95.04,42.400000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
med_df_transformed.isnull().sum()

Age                        0
Hypertension               0
Heart_Disease              0
Avg_Glucose_Lvl            0
BMI                        0
Stroke                     0
Gender_Male                0
Ever_Married_Yes           0
Work_Type_Govt_job         0
Work_Type_Never_worked     0
Work_Type_Private          0
Work_Type_Self-employed    0
Residence_Type_Urban       0
Smoker_Current             0
Smoker_Former              0
Smoker_Never               0
Smoker_Unknown             0
dtype: int64

In [12]:
# Create feature & target datasets
X = med_df_transformed.drop(columns=['Stroke'])
y = med_df_transformed['Stroke']
print(X.shape)
print(y.shape)

(1218, 16)
(1218,)


In [13]:
# Create empty arrays to store ML results
recalls = []
accuracies = []
precisions = []

# This will take a while. Feel free to adjust range to sample code. Feel free to review exported spreadsheets in github.
for i in range(1000):
    
    # Create training + testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    
    # Oversample positive stroke cases in training set
    ros = RandomOverSampler()
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    
    # Scale feature data
    scaler = StandardScaler()
    scaler.fit(X_train_resampled)
    X_train_scaled = scaler.transform(X_train_resampled)
    X_test_scaled = scaler.transform(X_test)
    
    # Create SVC model
    svc_model = SVC(kernel='linear')
    svc_model = svc_model.fit(X_train_scaled, y_train_resampled)
    
    # Evaluate SVC model
    y_pred = svc_model.predict(X_test_scaled)
    
    # Add recall results to list
    recalls.append(recall_score(y_test, y_pred))
    
    # Add precision results to list
    precisions.append(precision_score(y_test, y_pred))
    
    # Add accuracy results to list
    accuracies.append(accuracy_score(y_test, y_pred))
    
    print("Current progress:", i)

Current progress: 0
Current progress: 1
Current progress: 2
Current progress: 3
Current progress: 4
Current progress: 5
Current progress: 6
Current progress: 7
Current progress: 8
Current progress: 9
Current progress: 10
Current progress: 11
Current progress: 12
Current progress: 13
Current progress: 14
Current progress: 15
Current progress: 16
Current progress: 17
Current progress: 18
Current progress: 19
Current progress: 20
Current progress: 21
Current progress: 22
Current progress: 23
Current progress: 24
Current progress: 25
Current progress: 26
Current progress: 27
Current progress: 28
Current progress: 29
Current progress: 30
Current progress: 31
Current progress: 32
Current progress: 33
Current progress: 34
Current progress: 35
Current progress: 36
Current progress: 37
Current progress: 38
Current progress: 39
Current progress: 40
Current progress: 41
Current progress: 42
Current progress: 43
Current progress: 44
Current progress: 45
Current progress: 46
Current progress: 47
Cu

Current progress: 378
Current progress: 379
Current progress: 380
Current progress: 381
Current progress: 382
Current progress: 383
Current progress: 384
Current progress: 385
Current progress: 386
Current progress: 387
Current progress: 388
Current progress: 389
Current progress: 390
Current progress: 391
Current progress: 392
Current progress: 393
Current progress: 394
Current progress: 395
Current progress: 396
Current progress: 397
Current progress: 398
Current progress: 399
Current progress: 400
Current progress: 401
Current progress: 402
Current progress: 403
Current progress: 404
Current progress: 405
Current progress: 406
Current progress: 407
Current progress: 408
Current progress: 409
Current progress: 410
Current progress: 411
Current progress: 412
Current progress: 413
Current progress: 414
Current progress: 415
Current progress: 416
Current progress: 417
Current progress: 418
Current progress: 419
Current progress: 420
Current progress: 421
Current progress: 422
Current pr

Current progress: 751
Current progress: 752
Current progress: 753
Current progress: 754
Current progress: 755
Current progress: 756
Current progress: 757
Current progress: 758
Current progress: 759
Current progress: 760
Current progress: 761
Current progress: 762
Current progress: 763
Current progress: 764
Current progress: 765
Current progress: 766
Current progress: 767
Current progress: 768
Current progress: 769
Current progress: 770
Current progress: 771
Current progress: 772
Current progress: 773
Current progress: 774
Current progress: 775
Current progress: 776
Current progress: 777
Current progress: 778
Current progress: 779
Current progress: 780
Current progress: 781
Current progress: 782
Current progress: 783
Current progress: 784
Current progress: 785
Current progress: 786
Current progress: 787
Current progress: 788
Current progress: 789
Current progress: 790
Current progress: 791
Current progress: 792
Current progress: 793
Current progress: 794
Current progress: 795
Current pr

In [14]:
recalls

[0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0

In [15]:
recalls_df = pd.DataFrame(recalls, columns=['Recall'])
print(recalls_df.shape)
recalls_df.head(10)

(1000, 1)


,Recall
0,0.0
1,0.0
2,1.0
3,1.0
4,0.0
5,0.0
6,1.0
7,0.0
8,0.0
9,1.0


In [16]:
accuracies

[0.9098360655737705,
 0.9139344262295082,
 0.9221311475409836,
 0.9139344262295082,
 0.9590163934426229,
 0.9549180327868853,
 0.9262295081967213,
 0.9057377049180327,
 0.9344262295081968,
 0.9139344262295082,
 0.9016393442622951,
 0.9467213114754098,
 0.9180327868852459,
 0.9221311475409836,
 0.9016393442622951,
 0.9508196721311475,
 0.9262295081967213,
 0.8975409836065574,
 0.9672131147540983,
 0.9139344262295082,
 0.9262295081967213,
 0.8934426229508197,
 0.9344262295081968,
 0.9467213114754098,
 0.9016393442622951,
 0.9467213114754098,
 0.9262295081967213,
 0.9139344262295082,
 0.9221311475409836,
 0.9016393442622951,
 0.9262295081967213,
 0.9016393442622951,
 0.9549180327868853,
 0.9057377049180327,
 0.8934426229508197,
 0.9221311475409836,
 0.9139344262295082,
 0.9385245901639344,
 0.9057377049180327,
 0.9098360655737705,
 0.9385245901639344,
 0.9016393442622951,
 0.8934426229508197,
 0.9016393442622951,
 0.9508196721311475,
 0.9344262295081968,
 0.9057377049180327,
 0.9303278688

In [17]:
accuracies_df = pd.DataFrame(accuracies, columns=['Accuracy'])
print(accuracies_df.shape)
accuracies_df.head(10)

(1000, 1)


,Accuracy
0,0.909836
1,0.913934
2,0.922131
3,0.913934
4,0.959016
5,0.954918
6,0.926230
7,0.905738
8,0.934426
9,0.913934


In [18]:
precisions

[0.0,
 0.0,
 0.05,
 0.045454545454545456,
 0.0,
 0.0,
 0.05263157894736842,
 0.0,
 0.0,
 0.045454545454545456,
 0.04,
 0.0,
 0.0,
 0.05,
 0.04,
 0.0,
 0.05263157894736842,
 0.0,
 0.0,
 0.0,
 0.05263157894736842,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.04,
 0.0,
 0.041666666666666664,
 0.0,
 0.0,
 0.045454545454545456,
 0.0,
 0.041666666666666664,
 0.0,
 0.0,
 0.0,
 0.037037037037037035,
 0.0,
 0.0,
 0.0,
 0.041666666666666664,
 0.0,
 0.0,
 0.0,
 0.05,
 0.0,
 0.0,
 0.0,
 0.04,
 0.0,
 0.0,
 0.0,
 0.04,
 0.0,
 0.0,
 0.0,
 0.034482758620689655,
 0.047619047619047616,
 0.0,
 0.043478260869565216,
 0.0,
 0.041666666666666664,
 0.03571428571428571,
 0.03225806451612903,
 0.0,
 0.0,
 0.0,
 0.0,
 0.04,
 0.043478260869565216,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.05,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.045454545454545456,
 0.0,
 0.0,
 0.0,
 0.043478260869565216,
 0.0,
 0.041666666666666664,
 0.0,
 0.0,
 0.034482758620689655,
 0.045454545454545456,
 0.047619047619047616,
 0.0

In [19]:
precisions_df = pd.DataFrame(precisions, columns=['Precision'])
print(precisions_df.shape)
precisions_df.head(10)

(1000, 1)


,Precision
0,0.000000
1,0.000000
2,0.050000
3,0.045455
4,0.000000
5,0.000000
6,0.052632
7,0.000000
8,0.000000
9,0.045455


In [20]:
results_df = accuracies_df.merge(precisions_df, left_index=True, right_index=True)
print(results_df.shape)
results_df.head(10)

(1000, 2)


,Accuracy,Precision
0,0.909836,0.000000
1,0.913934,0.000000
2,0.922131,0.050000
3,0.913934,0.045455
4,0.959016,0.000000
5,0.954918,0.000000
6,0.926230,0.052632
7,0.905738,0.000000
8,0.934426,0.000000
9,0.913934,0.045455


In [21]:
results_df = results_df.merge(recalls_df, left_index=True, right_index=True)
print(results_df.shape)
results_df.head(10)

(1000, 3)


,Accuracy,Precision,Recall
0,0.909836,0.000000,0.0
1,0.913934,0.000000,0.0
2,0.922131,0.050000,1.0
3,0.913934,0.045455,1.0
4,0.959016,0.000000,0.0
5,0.954918,0.000000,0.0
6,0.926230,0.052632,1.0
7,0.905738,0.000000,0.0
8,0.934426,0.000000,0.0
9,0.913934,0.045455,1.0


In [22]:
print("Recall mean: " + str(results_df['Recall'].mean()))
print("Recall mode: " + str(results_df['Recall'].mode()))
print("Recall median: " + str(results_df['Recall'].median()))
print("Recall max: " + str(results_df['Recall'].max()))
print("Recall min: " + str(results_df['Recall'].min()))
print("Recall standard deviation: " + str(results_df['Recall'].std()))

Recall mean: 0.335
Recall mode: 0    0.0
dtype: float64
Recall median: 0.0
Recall max: 1.0
Recall min: 0.0
Recall standard deviation: 0.47222663838245754


In [23]:
print("Accuracy mean: " + str(results_df['Accuracy'].mean()))
print("Accuracy mode: " + str(results_df['Accuracy'].mode()))
print("Accuracy median: " + str(results_df['Accuracy'].median()))
print("Accuracy max: " + str(results_df['Accuracy'].max()))
print("Accuracy min: " + str(results_df['Accuracy'].min()))
print("Accuracy standard deviation: " + str(results_df['Accuracy'].std()))

Accuracy mean: 0.9262663934426212
Accuracy mode: 0    0.918033
dtype: float64
Accuracy median: 0.9221311475409836
Accuracy max: 0.9877049180327869
Accuracy min: 0.8647540983606558
Accuracy standard deviation: 0.022789922268144216


In [24]:
print("Precision mean: " + str(results_df['Precision'].mean()))
print("Precision mode: " + str(results_df['Precision'].mode()))
print("Precision median: " + str(results_df['Precision'].median()))
print("Precision max: " + str(results_df['Precision'].max()))
print("Precision min: " + str(results_df['Precision'].min()))
print("Precision standard deviation: " + str(results_df['Precision'].std()))

Precision mean: 0.015585789521718114
Precision mode: 0    0.0
dtype: float64
Precision median: 0.0
Precision max: 0.1
Precision min: 0.0
Precision standard deviation: 0.022596770772116928


In [25]:
results_df.isnull().sum().sum()

0

In [26]:
len(results_df)

1000

In [27]:
filepath = ('ML_result_files/SVC_GenY_SimpleImputer(mean)_RandomOverSampler.csv')
results_df.to_csv(filepath, index=False)